In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# =========================
# Files
# =========================
serial_file = "gwo_serial.csv"
openmp_file = "gwo_openmp.csv"
opencl_file = "gwo_opencl.csv"
cuda_file   = "gwo_cuda.csv"

out_dir = Path("plots_ndata")
out_dir.mkdir(exist_ok=True)

# =========================
# Load
# =========================
serial = pd.read_csv(serial_file)
openmp = pd.read_csv(openmp_file)
opencl = pd.read_csv(opencl_file)
cuda   = pd.read_csv(cuda_file)

# Types
for df in [serial, openmp, opencl, cuda]:
    df["Ndata"] = df["Ndata"].astype(int)
    df["POP_SIZE"] = df["POP_SIZE"].astype(int)
    df["avg_ms"] = df["avg_ms"].astype(float)

openmp["threads"] = openmp["threads"].astype(int)

# =========================
# Baseline map: (Ndata, POP_SIZE) -> serial avg_ms
# =========================
baseline = (
    serial[["Ndata","POP_SIZE","avg_ms"]]
    .rename(columns={"avg_ms":"T_serial_ms"})
    .copy()
)

# In case file has duplicates (same Ndata+POP), take min or mean; choose min to be safe
baseline = baseline.groupby(["Ndata","POP_SIZE"], as_index=False)["T_serial_ms"].min()

# Helper to join baseline
def attach_baseline(df: pd.DataFrame) -> pd.DataFrame:
    out = df.merge(baseline, on=["Ndata","POP_SIZE"], how="left")
    if out["T_serial_ms"].isna().any():
        miss = out[out["T_serial_ms"].isna()][["Ndata","POP_SIZE"]].drop_duplicates()
        raise ValueError("Missing serial baseline for:\n" + miss.to_string(index=False))
    out["speedup"] = out["T_serial_ms"] / out["avg_ms"]
    return out

# Karp–Flatt
def karp_flatt(speedup, p):
    s = np.asarray(speedup, dtype=float)
    p = np.asarray(p, dtype=float)
    e = (1/s - 1/p) / (1 - 1/p)
    e[p <= 1] = np.nan
    return e

# =========================
# Metrics
# =========================
openmp_m = attach_baseline(openmp)
openmp_m["efficiency"] = openmp_m["speedup"] / openmp_m["threads"]
openmp_m["karp_flatt_e"] = karp_flatt(openmp_m["speedup"], openmp_m["threads"])

opencl_m = attach_baseline(opencl)   # accelerator speedup vs serial (same Ndata, POP)
cuda_m   = attach_baseline(cuda)

openmp_m.to_csv("summary_openmp_metrics_ndata.csv", index=False)

# =========================
# 1) SERIAL stacked % per (Ndata) separately
# =========================
# Require these columns exist in your serial file
need_cols = [
    "avg_update_pop_excl_ms",
    "avg_update_fitness_excl_ms",
    "avg_fitness_batch_excl_ms",
    "avg_fitness_scalar_ms",
]
for c in need_cols:
    if c not in serial.columns:
        raise KeyError(f"serial missing column: {c}")

ser = serial.copy()
ser["t_update_pop_excl"] = ser["avg_update_pop_excl_ms"].astype(float)
ser["t_update_fitness_excl"] = ser["avg_update_fitness_excl_ms"].astype(float)
ser["t_fitness_batch_excl"] = ser["avg_fitness_batch_excl_ms"].astype(float)
ser["t_fitness_scalar"] = ser["avg_fitness_scalar_ms"].astype(float)

ser["t_accounted"] = (
    ser["t_update_pop_excl"] +
    ser["t_update_fitness_excl"] +
    ser["t_fitness_batch_excl"] +
    ser["t_fitness_scalar"]
)
ser["t_other"] = (ser["avg_ms"] - ser["t_accounted"]).clip(lower=0)

for c in ["t_update_pop_excl","t_update_fitness_excl","t_fitness_batch_excl","t_fitness_scalar","t_other"]:
    ser[c + "_pct"] = 100.0 * ser[c] / ser["avg_ms"]

for ndata in sorted(ser["Ndata"].unique()):
    sub = ser[ser["Ndata"] == ndata].sort_values("POP_SIZE")
    x = np.arange(len(sub))
    labels = [f"POP={p}" for p in sub["POP_SIZE"].tolist()]

    plt.figure(figsize=(12, 5))
    plt.title(f"Serial: % thời gian (exclusive) theo POP_SIZE | Ndata={ndata}")
    plt.xticks(x, labels)

    bottom = np.zeros(len(sub))
    plt.bar(x, sub["t_fitness_scalar_pct"], bottom=bottom, label="fitness_scalar (SSE core)")
    bottom += sub["t_fitness_scalar_pct"].to_numpy()

    plt.bar(x, sub["t_fitness_batch_excl_pct"], bottom=bottom, label="fitness_batch excl (overhead)")
    bottom += sub["t_fitness_batch_excl_pct"].to_numpy()

    plt.bar(x, sub["t_update_fitness_excl_pct"], bottom=bottom, label="update_fitness excl (heap/copy)")
    bottom += sub["t_update_fitness_excl_pct"].to_numpy()

    plt.bar(x, sub["t_update_pop_excl_pct"], bottom=bottom, label="update_pop excl (position update)")
    bottom += sub["t_update_pop_excl_pct"].to_numpy()

    plt.bar(x, sub["t_other_pct"], bottom=bottom, label="other / residue")

    plt.ylabel("Percent (%)")
    plt.ylim(0, 100)
    plt.grid(axis="y", linestyle="--", alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig(out_dir / f"serial_stacked_percent_ndata{ndata}.png", dpi=200)
    plt.close()

# =========================
# 2) OpenMP plots per (Ndata, POP)
# =========================
for ndata in sorted(openmp_m["Ndata"].unique()):
    subN = openmp_m[openmp_m["Ndata"] == ndata]
    for pop in sorted(subN["POP_SIZE"].unique()):
        sub = subN[subN["POP_SIZE"] == pop].sort_values("threads")
        if sub.empty:
            continue

        # speedup
        plt.figure()
        plt.plot(sub["threads"], sub["speedup"], marker="o")
        plt.xlabel("Threads")
        plt.ylabel("Speedup (vs Serial)")
        plt.title(f"OpenMP Speedup | Ndata={ndata}, POP={pop}")
        plt.xticks(sub["threads"].tolist())
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.tight_layout()
        plt.savefig(out_dir / f"openmp_speedup_ndata{ndata}_pop{pop}.png", dpi=160)
        plt.close()

        # efficiency
        plt.figure()
        plt.plot(sub["threads"], sub["efficiency"], marker="o")
        plt.xlabel("Threads")
        plt.ylabel("Efficiency")
        plt.title(f"OpenMP Efficiency | Ndata={ndata}, POP={pop}")
        plt.xticks(sub["threads"].tolist())
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.tight_layout()
        plt.savefig(out_dir / f"openmp_efficiency_ndata{ndata}_pop{pop}.png", dpi=160)
        plt.close()

        # karp-flatt
        plt.figure()
        plt.plot(sub["threads"], sub["karp_flatt_e"], marker="o")
        plt.xlabel("Threads")
        plt.ylabel("Karp–Flatt e")
        plt.title(f"OpenMP Karp–Flatt e | Ndata={ndata}, POP={pop}")
        plt.xticks(sub["threads"].tolist())
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.tight_layout()
        plt.savefig(out_dir / f"openmp_karpflatt_ndata{ndata}_pop{pop}.png", dpi=160)
        plt.close()

# =========================
# 3) Best framework compare per (Ndata, POP)
# =========================
# Best OpenMP by min avg_ms per (Ndata, POP)
best_openmp = openmp_m.loc[openmp_m.groupby(["Ndata","POP_SIZE"])["avg_ms"].idxmin()].copy()
best_openmp["framework"] = "OpenMP"
# best_openmp["label"] = best_openmp.apply(lambda r: f"OpenMP({int(r['threads'])})", axis=1)
best_openmp["label"] = best_openmp.apply(lambda r: f"OpenMP", axis=1)

# Best OpenCL by min avg_ms per (Ndata, POP)
best_opencl = opencl_m.loc[opencl_m.groupby(["Ndata","POP_SIZE"])["avg_ms"].idxmin()].copy()
best_opencl["framework"] = "OpenCL"
# best_opencl["label"] = best_opencl.apply(lambda r: f"OpenCL({r['device']})", axis=1)
best_opencl["label"] = best_opencl.apply(lambda r: f"OpenCL", axis=1)

# Best CUDA by min avg_ms per (Ndata, POP)
best_cuda = cuda_m.loc[cuda_m.groupby(["Ndata","POP_SIZE"])["avg_ms"].idxmin()].copy()
best_cuda["framework"] = "CUDA"
# best_cuda["label"] = best_cuda.apply(lambda r: f"CUDA({r['device']})", axis=1)
best_cuda["label"] = best_cuda.apply(lambda r: f"CUDA", axis=1)

best_compare = pd.concat([
    best_openmp[["Ndata","POP_SIZE","framework","avg_ms","speedup","label"]],
    best_opencl[["Ndata","POP_SIZE","framework","avg_ms","speedup","label"]],
    best_cuda[["Ndata","POP_SIZE","framework","avg_ms","speedup","label"]],
], ignore_index=True)

best_compare.to_csv("summary_best_framework_compare_ndata.csv", index=False)

# Bar per (Ndata, POP): speedup compare
for ndata in sorted(best_compare["Ndata"].unique()):
    subN = best_compare[best_compare["Ndata"] == ndata]
    for pop in sorted(subN["POP_SIZE"].unique()):
        sub = subN[subN["POP_SIZE"] == pop].sort_values("speedup", ascending=False)
        if sub.empty:
            continue

        plt.figure(figsize=(10, 4.5))
        plt.bar(sub["label"], sub["speedup"])
        plt.xticks(rotation=20, ha="right")
        plt.ylabel("Speedup vs Serial")
        plt.title(f"Best Speedup by Framework | Ndata={ndata}, POP={pop}")
        plt.grid(axis="y", linestyle="--", alpha=0.3)
        plt.tight_layout()
        plt.savefig(out_dir / f"best_framework_speedup_ndata{ndata}_pop{pop}.png", dpi=160)
        plt.close()

print("Saved:")
print(" - summary_openmp_metrics_ndata.csv")
print(" - summary_best_framework_compare_ndata.csv")
print(f" - plots in: {out_dir.resolve()}")


Saved:
 - summary_openmp_metrics_ndata.csv
 - summary_best_framework_compare_ndata.csv
 - plots in: D:\C_C++\Paralell_Programing\gwo-main\kmean_clustering\plots_ndata
